In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = 'C:\\Users\\user1\\Documents\\Sugandh\\Deep learning\\Project\\Split_smol\\train'
test_dir = 'C:\\Users\\user1\\Documents\\Sugandh\\Deep learning\\Project\\Split_smol\\val'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Assuming ResNet50 input size
    batch_size=32,
    class_mode='categorical'
)

Found 696 images belonging to 9 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 180 images belonging to 9 classes.


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(9, activation='softmax')(x)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Epoch 1/50
22/22 [==============================] - 25s 996ms/step - loss: 1.6171 - accuracy: 0.5043 - val_loss: 1.2231 - val_accuracy: 0.5722
Epoch 2/50
22/22 [==============================] - 22s 1s/step - loss: 0.9247 - accuracy: 0.6853 - val_loss: 1.0361 - val_accuracy: 0.6278
Epoch 3/50
22/22 [==============================] - 23s 1s/step - loss: 0.6723 - accuracy: 0.7443 - val_loss: 0.9865 - val_accuracy: 0.6722
Epoch 4/50
22/22 [==============================] - 22s 1000ms/step - loss: 0.7032 - accuracy: 0.7399 - val_loss: 0.8971 - val_accuracy: 0.6833
Epoch 5/50
22/22 [==============================] - 23s 1s/step - loss: 0.6206 - accuracy: 0.7773 - val_loss: 0.8779 - val_accuracy: 0.7000
Epoch 6/50
22/22 [==============================] - 21s 966ms/step - loss: 0.5990 - accuracy: 0.7845 - val_loss: 0.9597 - val_accuracy: 0.7056
Epoch 7/50
22/22 [==============================] - 22s 1s/step - loss: 0.5194 - accuracy: 0.8060 - val_loss: 1.0017 - val_accuracy: 0.6889
Epoch 8/50

In [ ]:
model.save('skin_disease_model_v6.h5')

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

6/6 [==============================] - 3s 540ms/step - loss: 1.2130 - accuracy: 0.7333
Test Accuracy: 73.33%
